CARGAR DATOS

In [6]:
import pandas as pd

# Ruta del archivo
ruta_archivo = 'C:/Users/dulce/Downloads/retail_sales_dataset.csv'

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv(ruta_archivo)

# Ver las primeras filas del DataFrame para confirmar la carga
print(df.head())

   Transaction ID        Date Customer ID  Gender  Age Product Category  \
0               1  2023-11-24     CUST001    Male   34           Beauty   
1               2  2023-02-27     CUST002  Female   26         Clothing   
2               3  2023-01-13     CUST003    Male   50      Electronics   
3               4  2023-05-21     CUST004    Male   37         Clothing   
4               5  2023-05-06     CUST005    Male   30           Beauty   

   Quantity  Price per Unit  Total Amount  
0         3              50           150  
1         2             500          1000  
2         1              30            30  
3         1             500           500  
4         2              50           100  


In [9]:
# Mostrar las columnas disponibles en el dataset
print(df.columns)

Index(['Transaction ID', 'Date', 'Customer ID', 'Gender', 'Age',
       'Product Category', 'Quantity', 'Price per Unit', 'Total Amount'],
      dtype='object')


TRANSFORMACIÓN DE DATOS

In [10]:
# Creación de la columna 'Ingreso Total' como cantidad vendida por precio unitario
df['Ingreso Total'] = df['Quantity'] * df['Price per Unit']

# Normalización de las ventas para la columna 'Ingreso Total'
df['Ventas Normalizadas'] = (df['Ingreso Total'] - df['Ingreso Total'].min()) / (df['Ingreso Total'].max() - df['Ingreso Total'].min())

# Verificar las primeras filas
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Ingreso Total,Ventas Normalizadas
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,150,0.063291
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,1000,0.493671
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,30,0.002532
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,500,0.240506
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,100,0.037975


In [11]:
# Función para clasificar el ingreso total en 'Alta', 'Media' o 'Baja'
def clasificar_ventas(valor):
    if valor > df['Ingreso Total'].quantile(0.75):
        return 'Alta'
    elif valor > df['Ingreso Total'].quantile(0.25):
        return 'Media'
    else:
        return 'Baja'

# Aplicar la función para crear la columna de clasificación
df['Clasificación de Ventas'] = df['Ingreso Total'].apply(clasificar_ventas)

# Verificar las primeras filas con la nueva clasificación
df[['Ingreso Total', 'Clasificación de Ventas']].head()

,Ingreso Total,Clasificación de Ventas
0,150,Media
1,1000,Alta
2,30,Baja
3,500,Media
4,100,Media


AGRUPACIÓN Y AGREGACIÓN

In [12]:
# Agrupación por 'Product Category' y 'Date' para estadísticas descriptivas
agrupado_product_date = df.groupby(['Product Category', 'Date']).agg({
    'Ingreso Total': ['sum', 'mean', 'count', 'min', 'max', 'std', 'var']
}).reset_index()

# Mostrar el resultado de la agrupación
agrupado_product_date.head()

Product Category        Date Ingreso Total                            \
                                         sum    mean count   min   max   
0           Beauty  2023-01-01          1500  1500.0     1  1500  1500   
1           Beauty  2023-01-02            25    25.0     1    25    25   
2           Beauty  2023-01-03           600   600.0     1   600   600   
3           Beauty  2023-01-04          1090   545.0     2    90  1000   
4           Beauty  2023-01-05            50    50.0     1    50    50   

                         
          std       var  
0         NaN       NaN  
1         NaN       NaN  
2         NaN       NaN  
3  643.467171  414050.0  
4         NaN       NaN

In [13]:
# Agrupación adicional por 'Product Category' y 'Gender'
agrupado_product_gender = df.groupby(['Product Category', 'Gender']).agg({
    'Ingreso Total': ['sum', 'mean', 'count', 'min', 'max', 'std', 'var']
}).reset_index()

# Mostrar el resultado de la agrupación adicional
agrupado_product_gender.head()

Product Category  Gender Ingreso Total                              \
                                     sum        mean count min   max   
0           Beauty  Female         74830  450.783133   166  25  2000   
1           Beauty    Male         68685  487.127660   141  25  2000   
2         Clothing  Female         81275  467.097701   174  25  2000   
3         Clothing    Male         74305  419.802260   177  25  2000   
4      Electronics  Female         76735  451.382353   170  25  2000   

                              
          std            var  
0  538.735040  290235.443593  
1  592.899725  351530.083587  
2  577.016491  332948.030862  
3  524.116237  274697.829995  
4  548.644646  301010.947616

ANÁLISIS PERSONALIZADO CON APPLY

In [14]:
# Función personalizada para calcular la desviación de cada venta respecto a la media del grupo
def calcular_desviacion(grupo):
    grupo['Desviación de la Media'] = grupo['Ingreso Total'] - grupo['Ingreso Total'].mean()
    return grupo

# Aplicar la función personalizada a grupos por 'Product Category' y 'Date'
df = df.groupby(['Product Category', 'Date']).apply(calcular_desviacion)

# Verificar las primeras filas con la desviación calculada
df[['Product Category', 'Date', 'Ingreso Total', 'Desviación de la Media']].head()

C:\Users\dulce\AppData\Local\Temp\ipykernel_23980\1155281830.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['Product Category', 'Date']).apply(calcular_desviacion)


Product Category        Date  Ingreso Total  \
Product Category Date                                                         
Beauty           2023-01-01 521           Beauty  2023-01-01           1500   
                 2023-01-02 978           Beauty  2023-01-02             25   
                 2023-01-03 609           Beauty  2023-01-03            600   
                 2023-01-04 31            Beauty  2023-01-04             90   
                            682           Beauty  2023-01-04           1000   

                                 Desviación de la Media  
Product Category Date                                    
Beauty           2023-01-01 521                     0.0  
                 2023-01-02 978                     0.0  
                 2023-01-03 609                     0.0  
                 2023-01-04 31                   -455.0  
                            682                   455.0

VERIFICACIÓN FINAL

In [15]:
# Mostrar las primeras filas del dataframe final
print("Primeras filas del dataframe transformado:")
print(df.head())

# Estadísticas descriptivas agrupadas por 'Product Category' y 'Date'
print("\nEstadísticas descriptivas agrupadas por 'Product Category' y 'Date':")
print(agrupado_product_date.head())

# Estadísticas descriptivas agrupadas por 'Product Category' y 'Gender'
print("\nEstadísticas descriptivas agrupadas por 'Product Category' y 'Gender':")
print(agrupado_product_gender.head())

Primeras filas del dataframe transformado:
                                 Transaction ID        Date Customer ID  \
Product Category Date                                                     
Beauty           2023-01-01 521             522  2023-01-01     CUST522   
                 2023-01-02 978             979  2023-01-02     CUST979   
                 2023-01-03 609             610  2023-01-03     CUST610   
                 2023-01-04 31               32  2023-01-04     CUST032   
                            682             683  2023-01-04     CUST683   

                                 Gender  Age Product Category  Quantity  \
Product Category Date                                                     
Beauty           2023-01-01 521    Male   46           Beauty         3   
                 2023-01-02 978  Female   19           Beauty         1   
                 2023-01-03 609  Female   26           Beauty         2   
                 2023-01-04 31     Male   30           B